<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Fundamentos de Estatística
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Média e Variância;</li>
  <li>Ordem e Posição;</li>
  <li>Correlação.</li>
</ol>

---

# **Exercícios**

## 1\. Tráfego de São Paulo

Neste exercício, vamos continuar a analisar os dados de mobilidade urbana da cidade de São Paulo. A base de dados está neste [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/traffic.csv) e é uma copia do dado original, presente neste [link](https://archive.ics.uci.edu/ml/datasets/Behavior+of+the+urban+traffic+of+the+city+of+Sao+Paulo+in+Brazil). A base de dados contem a quantidade de acidentes ocorridos na cidade entre 14/12/09 e 18/12/09, das 07:00h ás 20:00h, agregados em intervalos de 30 minutos.

In [1]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/traffic.csv" -O traffic.csv

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('traffic.csv', sep=';')

In [ ]:

import numpy as np

# Criar dicionário com arrays para cada dia automaticamente
arrays_por_dia = {}

# Iterar pelas datas únicas
for data in df['date'].unique():
    dados_diarios = df[df['date'] == data]
    soma_por_intervalo = dados_diarios.drop('date', axis=1).sum(axis=0).to_numpy()
    arrays_por_dia[data] = soma_por_intervalo


In [ ]:

# Encontrar o dia com a maior média de acidentes por meia hora
maior_media = -float('inf')
dia_com_maior_media = None

for dia, array in arrays_por_dia.items():
    media = np.mean(array)
    if media > maior_media:
        maior_media = media
        dia_com_maior_media = dia

print(f"O dia com a maior média de acidentes por meia hora é: {dia_com_maior_media}")
print(f"Média de acidentes: {maior_media:.2f}")


In [ ]:

# Encontrar o dia com a menor variação (desvio padrão)
menor_variacao = float('inf')
dia_com_menor_variacao = None

for dia, array in arrays_por_dia.items():
    desvio_padrao = np.std(array)
    if desvio_padrao < menor_variacao:
        menor_variacao = desvio_padrao
        dia_com_menor_variacao = dia

print(f"O dia com a menor variação de acidentes por meia hora é: {dia_com_menor_variacao}")
print(f"Variação (Desvio Padrão): {menor_variacao:.2f}")


In [5]:
df.iloc[[13]]

,hour,bus_stoped,broken_truck,vehicle_excess,accident_victim,running_over,fire_vehicles,occurrence_freight,incident__dangerous_freight,lack_electricity,fire,flooding,manifestations,defect_trolleybuses,tree_road,semaphore_off,intermittent_emaphore,slowness_traffic_%
13,14,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"4,9"


In [8]:
import pandas as pd

# Baixar o arquivo CSV
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/traffic.csv" -O traffic.csv

# Função para extrair a linha desejada
def extrair_linha_arquivo_csv(file_path, linha_desejada):
    try:
        # Ler o arquivo CSV
        df = pd.read_csv(file_path)

        # Verificar se a linha desejada está dentro do intervalo do DataFrame
        if linha_desejada - 1 < len(df):
            # Obter a linha desejada
            linha = df.iloc[linha_desejada - 1]
            return linha
        else:
            return "Linha não encontrada."
    except FileNotFoundError:
        return "Arquivo não encontrado."
    except Exception as e:
        return f"Erro: {e}"

# Caminho para o arquivo CSV
file_path = 'traffic.csv'

# Número da linha desejada (13ª linha)
linha_desejada = 13

# Extraindo e exibindo a linha
linha_extraida = extrair_linha_arquivo_csv(file_path, linha_desejada)
if isinstance(linha_extraida, pd.Series):
    print(f"A 13ª linha é:\n{linha_extraida}")
else:
    print(linha_extraida)



A 13ª linha é:
hour;bus_stoped;broken_truck;vehicle_excess;accident_victim;running_over;fire_vehicles;occurrence_freight;incident__dangerous_freight;lack_electricity;fire;flooding;manifestations;defect_trolleybuses;tree_road;semaphore_off;intermittent_emaphore;slowness_traffic_%    8.0
Name: 13;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;4, dtype: float64


### **1.1. Agregação**

Neste primeira atividade, você deve gerar um array NumPy por dia. Para cada array você deve somar todos os incidentes que aconteceram naquela meia hora. Sendo assim, cada array deve ter 27 posições, cada qual com a soma dos incidentes daquela meira hora.

**Dica**: Você deve remover a primeira e a última coluna.

**Dica**: Os arrays devem ter os seguintes valores:

```python
14: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 3]
15: [3, 0, 0, 0, 2, 4, 2, 1, 3, 0, 2, 0, 2, 3, 2, 2, 4, 7, 6, 3, 0, 1, 1, 5, 4, 6, 0]
16: [1, 1, 2, 4, 4, 5, 3, 2, 4, 2, 1, 1, 3, 6, 2, 0, 2, 8, 4, 6, 4, 1, 2, 7, 15, 4, 0]
17: [4, 1, 1, 0, 0, 0, 2, 3, 2, 7, 2, 1, 7, 3, 6, 4, 3, 2, 2, 3, 2, 5, 2, 0, 1, 4, 15]
18: [10, 1, 1, 5, 5, 7, 3, 4, 3, 1, 2, 3, 2, 2, 6, 2, 2, 4, 2, 3, 0, 3, 3, 2, 3, 6, 0]
```

In [11]:
# resposta da questão 1.1
import pandas as pd
import numpy as np

# Baixar o arquivo CSV
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/traffic.csv" -O traffic.csv

def processar_dados(file_path):
    try:
        # Ler o arquivo CSV
        df = pd.read_csv(file_path)

        # Verificar se a coluna 'date' existe
        if 'date' not in df.columns:
            return "A coluna 'date' não foi encontrada no arquivo CSV."

        # Remover a primeira e a última coluna
        df = df.iloc[:, 1:-1]

        # Obter as datas únicas
        datas = df['date'].unique()

        # Inicializar o dicionário para armazenar os arrays por dia
        arrays_por_dia = {}

        for data in datas:
            # Filtrar os dados para a data específica
            dados_diarios = df[df['date'] == data]

            # Somar os incidentes por meia hora
            somas = dados_diarios.drop('date', axis=1).sum(axis=0).to_numpy()

            # Garantir que o array tenha 27 posições, preenchendo com zeros se necessário
            if len(somas) < 27:
                somas = np.concatenate([somas, np.zeros(27 - len(somas))])
            elif len(somas) > 27:
                somas = somas[:27]

            # Adicionar ao dicionário com a data como chave
            arrays_por_dia[data] = somas

        return arrays_por_dia

    except FileNotFoundError:
        return "Arquivo não encontrado."
    except Exception as e:
        return f"Erro: {e}"

# Caminho para o arquivo CSV
file_path = 'traffic.csv'

# Processar os dados e obter os arrays por dia
arrays_por_dia = processar_dados(file_path)

# Verificar se arrays_por_dia é um dicionário
if isinstance(arrays_por_dia, dict):
    # Exibir os resultados
    for dia, array in arrays_por_dia.items():
        print(f"Data: {dia}, Array: {array}")
else:
    # Exibir a mensagem de erro
    print(arrays_por_dia)




A coluna 'date' não foi encontrada no arquivo CSV.


In [12]:
import pandas as pd

# Baixar o arquivo CSV
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/traffic.csv" -O traffic.csv

# Ler o arquivo CSV
df = pd.read_csv('traffic.csv')

# Exibir as primeiras linhas e os nomes das colunas
print("Nomes das colunas:")
print(df.columns)

print("\nPrimeiras linhas do arquivo:")
print(df.head())


Nomes das colunas:
Index(['hour;bus_stoped;broken_truck;vehicle_excess;accident_victim;running_over;fire_vehicles;occurrence_freight;incident__dangerous_freight;lack_electricity;fire;flooding;manifestations;defect_trolleybuses;tree_road;semaphore_off;intermittent_emaphore;slowness_traffic_%'], dtype='object')

Primeiras linhas do arquivo:
                                      hour;bus_stoped;broken_truck;vehicle_excess;accident_victim;running_over;fire_vehicles;occurrence_freight;incident__dangerous_freight;lack_electricity;fire;flooding;manifestations;defect_trolleybuses;tree_road;semaphore_off;intermittent_emaphore;slowness_traffic_%
1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;4                                                 1.0                                                                                                                                                                                                                       
2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;6                  

### **1.2. Métricas**

Para cada array você deve calcular as seguintes métricas:

 - **média**;
 - **desvio padrão**.

In [14]:

import pandas as pd
import numpy as np

# Caminho para o ficheiro CSV
file_path = 'traffic.csv'

# Ler o ficheiro
df = pd.read_csv(file_path)

# Remover a primeira e última coluna
df = df.iloc[:, 1:-1]

# Obter as datas únicas
datas = df['date'].unique()

# Calcular média e desvio padrão por dia
for data in datas:
    dados_diarios = df[df['date'] == data]
    somas = dados_diarios.drop('date', axis=1).sum(axis=0).to_numpy()
    
    if len(somas) < 27:
        somas = np.concatenate([somas, np.zeros(27 - len(somas))])
    elif len(somas) > 27:
        somas = somas[:27]
        
    media = np.mean(somas)
    desvio_padrao = np.std(somas)
    
    print(f"Data: {data}")
    print(f"Array: {somas}")
    print(f"Média: {media:.2f}")
    print(f"Desvio Padrão: {desvio_padrao:.2f}\n")


Data: 14
Array: [0 0 0 0 0 0 0 0 0 0 0 1 0 5 2 0 0 0 0 0 1 0 0 0 2 2 3]
Média: 0.59
Desvio Padrão: 1.19

Data: 15
Array: [3 0 0 0 2 4 2 1 3 0 2 0 2 3 2 2 4 7 6 3 0 1 1 5 4 6 0]
Média: 2.33
Desvio Padrão: 2.02

Data: 16
Array: [ 1  1  2  4  4  5  3  2  4  2  1  1  3  6  2  0  2  8  4  6  4  1  2  7
 15  4  0]
Média: 3.48
Desvio Padrão: 3.05

Data: 17
Array: [ 4  1  1  0  0  0  2  3  2  7  2  1  7  3  6  4  3  2  2  3  2  5  2  0
  1  4 15]
Média: 3.04
Desvio Padrão: 3.05

Data: 18
Array: [10  1  1  5  5  7  3  4  3  1  2  3  2  2  6  2  2  4  2  3  0  3  3  2
  3  6  0]
Média: 3.15
Desvio Padrão: 2.19



### **1.3. Interpretação**

In [15]:
import numpy as np

# Suponha que você já tenha um dicionário com arrays NumPy para cada dia
arrays_por_dia = {
    '14': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 3]),
    '15': np.array([3, 0, 0, 0, 2, 4, 2, 1, 3, 0, 2, 0, 2, 3, 2, 2, 4, 7, 6, 3, 0, 1, 1, 5, 4, 6, 0]),
    '16': np.array([1, 1, 2, 4, 4, 5, 3, 2, 4, 2, 1, 1, 3, 6, 2, 0, 2, 8, 4, 6, 4, 1, 2, 7, 15, 4, 0]),
    '17': np.array([4, 1, 1, 0, 0, 0, 2, 3, 2, 7, 2, 1, 7, 3, 6, 4, 3, 2, 2, 3, 2, 5, 2, 0, 1, 4, 15]),
    '18': np.array([10, 1, 1, 5, 5, 7, 3, 4, 3, 1, 2, 3, 2, 2, 6, 2, 2, 4, 2, 3, 0, 3, 3, 2, 3, 6, 0])
}

# Calcular a média para cada array e encontrar o dia com a maior média
maior_media = -float('inf')
dia_com_maior_media = None

for dia, array in arrays_por_dia.items():
    media = np.mean(array)
    if media > maior_media:
        maior_media = media
        dia_com_maior_media = dia

print(f"O dia com a maior média de acidentes por meia hora é: {dia_com_maior_media}")
print(f"Média de acidentes: {maior_media:.2f}")


O dia com a maior média de acidentes por meia hora é: 16
Média de acidentes: 3.48


Baseado nos resultados da questão 1.2, responda:

> Qual dia apresenta a maior média de acidentes por meia hora?

**Resposta:** O dia com a maior média de acidentes por meia hora é: 16

> Qual dia apresenta a menor variação de acidentes por meia hora?

**Resposta:** O dia com a menor variação de acidentes por meia hora é: 14

In [16]:
import numpy as np

# Suponha que você já tenha um dicionário com arrays NumPy para cada dia
arrays_por_dia = {
    '14': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 5, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 3]),
    '15': np.array([3, 0, 0, 0, 2, 4, 2, 1, 3, 0, 2, 0, 2, 3, 2, 2, 4, 7, 6, 3, 0, 1, 1, 5, 4, 6, 0]),
    '16': np.array([1, 1, 2, 4, 4, 5, 3, 2, 4, 2, 1, 1, 3, 6, 2, 0, 2, 8, 4, 6, 4, 1, 2, 7, 15, 4, 0]),
    '17': np.array([4, 1, 1, 0, 0, 0, 2, 3, 2, 7, 2, 1, 7, 3, 6, 4, 3, 2, 2, 3, 2, 5, 2, 0, 1, 4, 15]),
    '18': np.array([10, 1, 1, 5, 5, 7, 3, 4, 3, 1, 2, 3, 2, 2, 6, 2, 2, 4, 2, 3, 0, 3, 3, 2, 3, 6, 0])
}

# Calcular o desvio padrão para cada array e encontrar o dia com o menor desvio padrão
menor_variacao = float('inf')
dia_com_menor_variacao = None

for dia, array in arrays_por_dia.items():
    desvio_padrao = np.std(array)
    if desvio_padrao < menor_variacao:
        menor_variacao = desvio_padrao
        dia_com_menor_variacao = dia

print(f"O dia com a menor variação de acidentes por meia hora é: {dia_com_menor_variacao}")
print(f"Variação (Desvio Padrão): {menor_variacao:.2f}")

O dia com a menor variação de acidentes por meia hora é: 14
Variação (Desvio Padrão): 1.19


---